Clara Liff

In [1]:
#1
import scipy.io
import scipy.stats
import numpy as np
import matplotlib.pyplot as plt
import pickle

In [ ]:
#a
def calcfanofac(M,tstart,tend):
    S = np.sum(M[:,tstart:tend],axis=1)
    FF = np.var(S)/np.mean(S)
    
    return FF

In [ ]:
#b
PMDdata = pickle.load(open("PMDdata.pkl","rb"))
N = len(PMDdata)
# time bins
pre_start = 300
pre_end = 400
post_start = 500
post_end = 600
# Fano factors for each sample
fano_pre = np.zeros(N)
fano_post = np.zeros(N)
# loop over samples, get spikes and compute Fano factors
for ii in range(N):
    M = PMDdata[ii]
    fano_pre[ii] = calcfanofac(M,pre_start,pre_end)
    fano_post[ii] = calcfanofac(M,post_start,post_end)
# ignore samples that have NaN values
valid_inds = ~(np.isnan(fano_pre) | np.isnan(fano_post))
fano_pre = fano_pre[valid_inds]
fano_post = fano_post[valid_inds]

In [ ]:
#c
fano_pre_avg = np.mean(fano_pre)
fano_post_avg = np.mean(fano_post)

print('fano_pre average: '+str(fano_pre_avg)+'\nfano_post average: '+str(fano_post_avg))
counts, bins = np.histogram(fano_pre)
plt.stairs(counts, bins, label='fano_pre');
counts, bins = np.histogram(fano_post)
plt.stairs(counts, bins, label='fano_post');
plt.title('Fano factors before and after reach')
plt.xlabel('Fano factor')
plt.ylabel('frequency')
plt.legend();
# You can use a paired test because the measurements are from the same cells
# Wilcoxon signed-rank test, p-value = 1.4955603795016292e-148
scipy.stats.wilcoxon(fano_pre,fano_post)

In [ ]:
#2
#a
J = np.load('kcs.npz')["J"]
conns = np.sum(J,1); # connections per Kenyon cell
N_KC = np.size(J,0) # number of Kenyon cells
N_PN = np.size(J,1) # number of presynaptic projection neurons
conn_prob = np.mean(J) # connection probability

edges = np.arange(12)
K_data = np.histogram(conns,edges)[:11][0]
Ntrials=1000
K_shuffle = np.zeros([Ntrials,11])
for ti in range(Ntrials):
    J_shuffle = np.random.rand(N_KC,N_PN) < conn_prob
    conns_shuffle = np.sum(J_shuffle,1)
    K_shuffle[ti,:] = np.histogram(conns_shuffle,edges)[:11][0]

In [ ]:
#b
pvals = np.zeros(11)
for i in range(11):
    greater_equal = len(K_shuffle[K_shuffle[:,i] >= K_data[i]])/Ntrials
    less_equal = len(K_shuffle[K_shuffle[:,i] <= K_data[i]])/Ntrials
    min_fraction = min(greater_equal,less_equal)
    pvals[i] = min_fraction
print(pvals)

In [ ]:
#c
alpha = 0.05/11 # to account for multiple comparisons
for i in range(len(pvals)):
    if pvals[i] < alpha:
        print('k='+str(i))